In [1]:
# -*- coding: utf-8 -*-
#Tanasorn Chindasook 
#Web crawler for exchange rates, advanced projects 1 Professor Adalbert Wilhelm

import urllib2
from urllib2 import urlopen
from bs4 import BeautifulSoup as soup
import numpy as np
import datetime
import time
import pandas as pd
import re
import sys
reload(sys)
sys.setdefaultencoding("utf-8")
import requests

In [2]:
#read csv for dates that need to be crawled
csv_data = pd.read_csv("exchange_dates.csv")
#remove id column from r generated file
csv_data.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [3]:
#Get all possible currencies needed to be converted
currencies = csv_data['positions_currency'].unique()

#get all possible dates needed to be crawled and replace all slashes with dashes
dates = csv_data['created_date'].str.replace("/","-").unique()

In [4]:
currencies

array(['PLN', 'MXN', 'GBP', 'CHF', 'CAD', 'USD', 'BRL', 'SEK'],
      dtype=object)

In [5]:
#reformat the dates to be used in url 
new_dates = []
for date in dates:
    date_dict = {}
    split_dates = date.split("-")
    date_dict.update({'d':split_dates[0], 'm':split_dates[1],'y':split_dates[2]})
    new_dates.append(date_dict)

In [6]:
historical_exchange_rates = []

In [7]:
for i in range(len(new_dates)):
    #form the url by the new dates and make request
    time.sleep(7)
    url = 'https://www.xe.com/currencytables/?from=EUR&date=' + str(new_dates[i]['y']) + '-' + str(new_dates[i]['m']) + '-' +str(new_dates[i]['d'])

    r = requests.get(url, verify=False)
    
    
    page_html = r.text
    page_soup = soup(page_html,"html.parser")
    
    containers = page_soup.findAll('tr')
    
    #get each exchange rate
    usd = containers[1].findAll('td')[3].text
    cad = containers[6].findAll('td')[3].text
    chf = containers[8].findAll('td')[3].text
    mxn = containers[17].findAll('td')[3].text
    sek = containers[20].findAll('td')[3].text
    gbp = containers[3].findAll('td')[3].text
    brl = containers[23].findAll('td')[3].text
    pln = containers[35].findAll('td')[3].text

    #create dict with dates
    exchange_rates = {'date':dates[i]}
    exchange_rates.update({'USD':usd,"CAD":cad,"CHF":chf,"MXN":mxn,"SEK":sek,"GBP":gbp,"PLN":pln})
    historical_exchange_rates.append(exchange_rates)



In [8]:
df = pd.DataFrame(historical_exchange_rates)
df.to_csv("historical_exchange_rates1.csv")